# Sudo Extrinsic Consolidation



## 0. Imports

In [1]:
import json
from collections import defaultdict, Counter
from substrateinterface import ss58_decode, ss58_encode

## 1. Extract Extrinsic Calls

In [2]:
with open('sudo-extrinsics.json') as sudo_ext:
    sudo_ext_calls = json.load(sudo_ext)
    sudo_ext_calls.reverse()

## 2. Remove Failed Calls

In [3]:
passing_sudo_ext_calls = [x for x in sudo_ext_calls if x['success']]

print(f"Removed {len(sudo_ext_calls) - len(passing_sudo_ext_calls)} failed calls. ({len(passing_sudo_ext_calls)}/{len(sudo_ext_calls)}) remaining.")

Removed 22 failed calls. (928/950) remaining.


In [4]:
len(passing_sudo_ext_calls)

928

## 3. Flatten batch_all Calls

In [5]:
def extract_calls(ext):
    if ext['value']['call_name'] == 'force_update_schedules':
        return [ext['value']]
    elif ext['value']['call_name'] == 'force_vested_transfer':
        return [ext['value']]
    elif ext['value']['call_name'] == 'batch_all':
        return ext['value']['params'][0]['value']
        
    else:
        #print(f"Unexpected call name: {ext['value']['call_name']}")
        return []

In [6]:
only_sudo_call = [extract_calls(json.loads(x['params'])[0]) for x in sudo_ext_calls if x['call_module_function'] == 'sudo']

only_sudo_call_flat = []
_ = [only_sudo_call_flat.extend(x) for x in only_sudo_call]

In [7]:
len(only_sudo_call_flat)

86220

In [8]:
only_sudo_call_flat[85000]

{'call_index': '2004',
 'call_module': 'Vesting',
 'call_name': 'force_update_schedules',
 'params': [{'name': 'target',
   'type': 'sp_runtime:multiaddress:MultiAddress',
   'value': {'Id': '0x089f1a8d0514da8150aa20a136e1575254c9f1b0d5b464b8c520ba5490838124'}},
  {'name': 'schedules',
   'type': 'Vec<pallet_vesting:vesting_info:VestingInfo@153>',
   'value': [{'locked': '5019200000000000000000',
     'per_block': '1394222222222222222',
     'starting_block': 207800},
    {'locked': '45172800000000000000000',
     'per_block': '32010204081632653',
     'starting_block': 210541}]}]}

## 4. Additional Filtering of Vesting Calls

In [9]:
set([x['call_name'] for x in only_sudo_call_flat])

{'developer_pre_approval',
 'force_transfer',
 'force_update_schedules',
 'force_vested_transfer'}

In [10]:
vesting_calls = ['force_update_schedules', 'force_vested_transfer']

only_vesting_calls = [x for x in only_sudo_call_flat if x['call_name'] in vesting_calls]

In [11]:
len(only_vesting_calls)

84540

In [12]:
len([x for x in only_vesting_calls if x['call_name'] == 'force_update_schedules'])

2483

## 5. Consolidate Vesting Sequences

In [13]:
only_vesting_calls[0]

{'call_index': '2003',
 'call_module': 'Vesting',
 'call_name': 'force_vested_transfer',
 'params': [{'name': 'source',
   'type': 'sp_runtime:multiaddress:MultiAddress',
   'value': {'Id': '0xe8cd33672da831212e839cfebbed77d1e9c113f98522217468bca1cdb1ee124b'}},
  {'name': 'target',
   'type': 'sp_runtime:multiaddress:MultiAddress',
   'value': {'Id': '0x1ae4acd9ca0dc34276b20c26bd11e6932419ca7bceb66601a1483b2c95038514'}},
  {'name': 'schedule',
   'type': 'pallet_vesting:vesting_info:VestingInfo@149',
   'value': {'locked': '1413742000000000000',
    'per_block': '318754960317',
    'starting_block': 210541}}]}

In [14]:
only_vesting_calls[0]['params'][1]['value']['Id']

'0x1ae4acd9ca0dc34276b20c26bd11e6932419ca7bceb66601a1483b2c95038514'

In [15]:
only_vesting_calls[0]['params'][2]

{'name': 'schedule',
 'type': 'pallet_vesting:vesting_info:VestingInfo@149',
 'value': {'locked': '1413742000000000000',
  'per_block': '318754960317',
  'starting_block': 210541}}

In [16]:
length_fun = lambda x: round(int(x['params'][2]['value']['locked']) / int(x['params'][2]['value']['per_block']))

length_fun_2 = lambda x: [round(int(schedule['locked']) / int(schedule['per_block'])) for schedule in x['params'][1]['value']]

vesting_durations = list(map(length_fun, [x for x in only_vesting_calls if x['call_name'] == 'force_vested_transfer']))

Counter(vesting_durations)

Counter({4435200: 34174,
         4838400: 1,
         3600: 41403,
         1411200: 1451,
         3024000: 5025,
         2016000: 3})

Judging from this:
* 1411200 ==> 6.53 months so this should be lockdrop 1st tier
* 3024000 ==> 14 months so this should be lockdrop 2nd tier

In [17]:
vesting_durations_2 = list(map(length_fun_2, [x for x in only_vesting_calls if x['call_name'] == 'force_update_schedules']))
vesting_durations_2_flat = []
_ = [vesting_durations_2_flat.extend(x) for x in vesting_durations_2]

Counter(vesting_durations_2_flat)

Counter({1: 22,
         1411200: 1335,
         1209600: 1226,
         3600: 1126,
         4435200: 69,
         3024000: 10})

In [18]:
FIRST_TIER_LD = 1411200
SECOND_TIER_LD = 3024000

FIRST_TIER_LD_REDUCED = 1209600 # for the ones that were modified?

LOCKDROP_TIERS = set([FIRST_TIER_LD, SECOND_TIER_LD, FIRST_TIER_LD_REDUCED])

In [19]:
tmp = [x for x in only_vesting_calls if x['call_name'] == 'force_update_schedules']
tmp[50]

{'call_index': '2004',
 'call_module': 'Vesting',
 'call_name': 'force_update_schedules',
 'params': [{'name': 'target',
   'type': 'sp_runtime:multiaddress:MultiAddress',
   'value': {'Id': '0x0e471e3c4254e3e280c554f84c58d300b9551dc7cb3fb03c641f4bd658449fbd'}},
  {'name': 'schedules',
   'type': 'Vec<pallet_vesting:vesting_info:VestingInfo@149>',
   'value': [{'locked': '47375991000000000000000',
     'per_block': '39166659226190476',
     'starting_block': 210541}]}]}

In [20]:
on_chain_vesting_info = defaultdict(list)

SS58_FORMAT = 5

for x in only_vesting_calls:
    if x['call_name'] == 'force_vested_transfer':
        address = x['params'][1]['value']['Id']
        address = ss58_encode(address, SS58_FORMAT)
        on_chain_vesting_info[address].append(length_fun(x))
                                              
    elif x['call_name'] == 'force_update_schedules':
        address = x['params'][0]['value']['Id']
        address = ss58_encode(address, SS58_FORMAT)
        print(f"For {address} replacing {on_chain_vesting_info[address]} with {length_fun_2(x)}")      
        
        on_chain_vesting_info[address] = length_fun_2(x)
    else:
        print("ERROR! Shouldn't happen!")

For VwJHxi1vTNdnJ2KXfQ7tWL8ERiUGv9bwhmf5RDWKGySQmsy replacing [1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200, 1411200] with [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
For bNMHadaUHNhH8Nh5EANTkm1fw3Vdp4ysVrcrZ1S811xa1oC replacing [1411200] with [1411200]
For WavRZKdCNd1jsaFcrMVBMAU6gdMynjjisVV4N28yVFTHgsK replacing [1411200] with [1411200]
For X5tTtn5bSu3f3xhUoreERztDdnLRAMg7juRMyeecTbjniv1 replacing [1411200] with [1411200]
For WJN2GyYeKXM6pHJqqZzxautW7igSRfsYqNWLWJ3HA7dhBLM replacing [1411200] with [1411200]
For YvZEF4GqvdkcTChpATgrPKF2SZLSnoUfuykE7sw7D3jrTos replacing [1411200] with [1411200]
For XzSLA3aotF4sXiHW8c8Pm7fwptZ4TMSNGL1TfhTvhYWFzhH replacing [1411200] with [1411200]
For XQ5mBRMhkZ31AwztWM3QR7aqPyk8jvUqMAuh718S2qRs1Xv replacing [1411200] with [1411200]
For b1nYHE2F1MN6VxpVaH58Rg5PjjmTXduP54CNqkYmqgpK7ET replac

In [21]:
len(on_chain_vesting_info)

39244

In [22]:
only_first_second_tier = dict()
double_vestings_accounts = dict()

for (k, v) in on_chain_vesting_info.items():
    filtered_list = list(filter(lambda x: x in LOCKDROP_TIERS, v))
    #filtered_list = v
    if len(filtered_list) > 0:
        only_first_second_tier[k] = filtered_list
    if len(filtered_list) >= 2:
        double_vestings_accounts[k] = filtered_list
        #print(f"{k} has multiple lockdrop vestings: {filtered_list}")
        
double_vestings_accounts

{'YpvhUm4k3jAd8iEPdjZxBW8S1E7Qu3FGC1boEEh9PbTRSoS': [3024000, 1411200],
 'WSg41PPaPE6VBVyen3i7wBy1Mcm5rAaN66bVorFcQpbnHdV': [1411200, 3024000],
 'YuWy9gMKupdbnG6zvWNWkmqcb4hpma9uw2m3EJaXj2dnndG': [3024000, 1411200],
 'Z1LiAR3sSWg1VV5CxdtqKVczFfmZegRCRHw4MzRVuLMnkrv': [3024000, 1411200],
 'W24UAJMr8XwjudPDGAxT46CE3GJgiMqdjZsYoVAcME4EUT3': [1411200, 3024000],
 'YLgHfn6BEoa6g7TiHkXgeyEnipVcGzUDCgdqHKbM4fcp77y': [3024000, 1411200],
 'ZBLwT5AMhuSoEqMGRAnj6T8T1FiyfGqeZWVAnbcLEsKxCmj': [1411200, 3024000],
 'b3AAiA1cVvafPx6jT6MUDPWvcVLHGpqm3o2TNbLNQAbRdUt': [3024000, 1411200],
 'ZLd5Fs8Aq8xLbZZneMEgLTcqt2edWKLXxE9sL1BUWtAa4HX': [3024000, 1411200],
 'XfXDYRCSUZh1phzaLRKp2bwzyJGEfdigrycZecnNdLpbcmY': [3024000, 1411200],
 'WSSdn1UBGytofBGC41HXgvWTNAGV4gMaTmf3D7aDwgzPXnD': [1411200, 1411200]}

## 6. Compare Data

In [23]:
with open('plasm_to_duration.json') as ptd:
    plasm_to_duration_from_ld = json.load(ptd)
#plasm_to_duration_from_ld = {ss58_encode('0x' + ss58_decode(k), SS58_FORMAT): v for (k, v) in plasm_to_duration_from_ld.items() }
    
plasm_to_duration_from_chain = only_first_second_tier


In [24]:
len(plasm_to_duration_from_ld.keys())

5985

In [25]:
len(plasm_to_duration_from_chain.keys())

6453

In [26]:
len(set(plasm_to_duration_from_chain.keys()).intersection(set(plasm_to_duration_from_ld.keys())))

2633

In [27]:
LONG_VESTING = [3024000]
SHORT_VESTING = [1411200, 1209600]

missing_addresses = set()

for (addr, durations) in plasm_to_duration_from_chain.items():
    if addr not in plasm_to_duration_from_ld.keys():
        missing_addresses.add(addr)
        continue # TODO: This is a huge problem since 3k+ mappings are missing
        
    if len(durations) > 1:
        pass # TODO Imo, this can be ignored
    else:   
        is_long_on_chain_duration = durations[0] in LONG_VESTING
        is_long_ld_duration = plasm_to_duration_from_ld[addr] < 1000
        
        if is_long_ld_duration != is_long_on_chain_duration:
            print(f"Address {addr} has incorrect vesting duration! Has {durations[0]} blocks but lockdrop duration was {plasm_to_duration_from_ld[addr]}.")

Address ZeWq35cxECxvxL1naf5v1B5e2uzq7iuBCujmEdQXB3n6fzc has incorrect vesting duration! Has 1411200 blocks but lockdrop duration was 300.
Address WHqko9UZqBGv4dUihjVuMWkjMXdkPH5HPSTDdKowjVpSvzQ has incorrect vesting duration! Has 1411200 blocks but lockdrop duration was 100.
Address ayuBUfi7eSVsrRaVCxTse9XATRi38QK33teFsZNh6KnqASN has incorrect vesting duration! Has 1411200 blocks but lockdrop duration was 100.
Address ZeCHeRvLbjgQoNAMHUvjfk84EJNaHPirsZUsxA4HJ3fcUCA has incorrect vesting duration! Has 1411200 blocks but lockdrop duration was 300.
Address YYou8iYYk298sNwNsQd9ftcEGsGKuXy2nsk1K4xNpkXcnyb has incorrect vesting duration! Has 1411200 blocks but lockdrop duration was 100.
Address WX78pckuXBqdtXQafpVhwcWRUfdH47mF7QcNyKK2MLmo2PT has incorrect vesting duration! Has 3024000 blocks but lockdrop duration was 1000.
Address WHXEjz8EA1tVS1Jtvw65WhJ1JHDMjheeMeJWpgvovtFFh15 has incorrect vesting duration! Has 3024000 blocks but lockdrop duration was 1000.
Address bX53u78nVAG12Zz8R1YZrNVa

In [28]:
len(missing_addresses)

3820

In [29]:
for x in missing_addresses:
    print(f"{x} ==> {plasm_to_duration_from_chain[x]}")

bKn5be5Mo6S8YakBib6TMnEfnwBKNhSVTFfJZBJAeKQN4fU ==> [3024000]
YPhpJv17azZx2ar6EETGhxXVh8yEq4mcUwujgAna1Kr1sLX ==> [3024000]
WMjpyoejizMfeBNymED2pmkLCAtiBLCBE1ewkGF6it9ZRys ==> [1411200]
YJovTSwTMB9Z1QNZeB8f5HQq9jhXW3v8azWBU3osxSFMqVH ==> [1411200]
aLkv4cJgamaw14Xq67RUwvbzUbdyxMQhEfSuNF892BdMKWy ==> [1411200]
Y4kPBLG6HcN9aXyycnJvj4wWhYtMpcvE8UjL98EtW6jzjF8 ==> [1411200]
XXwc2jUmv7hyDt6swrx8yGaZgNiPBLMsBLHEGAaxypbjm95 ==> [3024000]
Z5NXk9Yg7uBQnjnGDXzWHBqQCd78QgQvfjqVcF6hvxx9KyT ==> [1411200]
ZkrtuYVfq18gqspyzZmu3MoXR3V3eAMr55yt7nh6ogpGPNx ==> [3024000]
bb6MmKDQvsgvAZHgdep1Sh2KpNG8frL3e2zsRpw6dpuQ52V ==> [1411200]
bNpKkUCQpsxEgQ4Bd91gTDXVL5QmdcqueHrm6SHB364L6Zo ==> [1411200]
bLMZp8gDFiMCxXoRzMnkBJ9Un8h6Kr3XSmez3QB9GeTj5gZ ==> [3024000]
WSyQCtgxV2fGUmzCcidBE8bA342Apb93zuix8BgxEerGHkG ==> [3024000]
W5YMSeF2j31Gszsnhy2GWjLftA8rUef8Yp7MUHjFPzg4wJZ ==> [1411200]
YoZ3AY8ktfp5A6QtVtHT2EAbPo91xH32xGFuEq4CNgzwngE ==> [3024000]
XdbzUTRQGoF62MZ2AAcQPivPs5K2BtS2rUxvJ43jbpSLhhu ==> [1411200]
a295UWx5